In [3]:
import numpy
import matplotlib.pyplot as plt

In [4]:
data=numpy.load(r'..\间隔_5min_init.npy',allow_pickle=True).item()
info=numpy.load(r'..\路段信息_简略.npy',allow_pickle=True).item()

In [1]:
normal_days=[2, 3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 28, 29, 30, 31, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 49, 50, 51, 52, 55, 56, 57, 58, 59] #除去周末和假日

In [2]:
len(normal_days)

41

In [9]:
targets=[139,316,46,47,138,33,34,45,246,385,42,37,41,43,44,152] #目标路段
train={}
test={}
for seg in targets:#从数据集中挑选出训练集、测试集部分
    train[seg]=[data[seg][i] for i in normal_days[:33]]
    test[seg]=[data[seg][i] for i in normal_days[33:]]

统计各路段拥堵状况（时速小于20公里/时）的比例

In [13]:
valid_seg=[139,316,46,47,138,33,34,45,246,385,42,37,41,43,44,152]
for seg in valid_seg:
    seg_len=float(info[str(seg)][5])
    len_arr=numpy.array([seg_len for i in range(41*180)])
    len_arr=len_arr.reshape((41,180))
    speed=numpy.around(len_arr*3.6/train[seg],2)
    # print(numpy.std(speed))
    # mean_speed=numpy.around(numpy.mean(speed),2)
    low_speed_num=numpy.sum(numpy.where(speed<20,1,0))
    print(seg,low_speed_num/41/180*100)

139 5.858585858585859
316 7.121212121212121
46 51.936026936026934
47 30.976430976430976
138 3.4848484848484853
33 76.95286195286195
34 56.64983164983165
45 29.208754208754208
246 93.77104377104376
385 26.481481481481485
42 85.53872053872054
37 33.08080808080808
41 55.94276094276094
43 58.16498316498316
44 1.4141414141414141
152 56.78451178451178


In [6]:
def get_ewa(series,look_back_span): #指数移动平均法
    ewa=[numpy.mean(series[:look_back_span])]
    index=0
    weight=2/(1+look_back_span)
    for value in series[look_back_span:]:
        ewa.append(value*weight+ewa[index]*(1-weight))
        index+=1
    return ewa

分别对测试集、训练集进行数据的平滑操作

In [13]:
for seg in targets:
    tmp=[get_ewa(day,4) for day in train[seg]]
    train[seg]=tmp

In [14]:
for seg in targets:
    tmp=[get_ewa(day,4) for day in test[seg]]
    test[seg]=tmp

## 归一化

In [15]:
max_min_time={}#用于记录各路段的最大最小旅行时间

In [16]:
train_days,train_dim=(len(train[targets[0]]),len(train[targets[0]][0]))
test_days,test_dim=(len(test[targets[0]]),len(test[targets[0]][0]))
train_new={}
test_new={}
for seg in targets:
    train_arr=numpy.array(train[seg])
    test_arr=numpy.array(test[seg])
    max_train=numpy.max(train_arr)
    min_train=numpy.min(train_arr) 
    max_min_time[seg]=[max_train,min_train]
    bias=(max_train-min_train)
    if max_train==min_train:
        print(seg)
        # pass
    min_test_arr=[]
    for i in range(test_days):
        min_test_arr.append([min_train]*test_dim)
    min_train_arr=[]
    for i in range(train_days):
        min_train_arr.append([min_train]*train_dim)

    min_train_arr=numpy.array(min_train_arr)#array化，从而可以利用numpy库加快运算
    min_test_arr=numpy.array(min_test_arr)#array化

    train_arr=numpy.around((train_arr-min_train_arr)/bias,2)
    test_arr=numpy.around((test_arr-min_test_arr)/bias,2)
    # 归一化后的训练、测试集
    train_new[seg]=train_arr
    test_new[seg]=test_arr

In [17]:
max_min_time

{139: [188.0193977142086, 35.51176541357626],
 316: [236.61108834696307, 36.0],
 46: [144.1258795495733, 38.0],
 47: [154.8285546865203, 38.0],
 138: [219.57448679936002, 64.36609612032],
 33: [306.41881613393923, 25.0],
 34: [153.79233010481434, 25.0],
 45: [285.9322907357116, 23.0],
 246: [381.04258732817493, 23.0],
 385: [172.97847789126544, 68.0],
 42: [150.05058080107455, 34.0],
 37: [455.0680911350173, 41.0],
 41: [294.1113390332663, 34.0],
 43: [144.99871176726572, 23.0],
 44: [86.41231798983719, 23.0],
 152: [406.50166033716033, 68.0]}

In [19]:
train_new[43][1][:5]

array([0.18, 0.16, 0.15, 0.16, 0.18])

In [20]:
numpy.save(r'train.npy',train_new)
numpy.save(r'test.npy',test_new)

In [10]:
mean_mat={} #记录各路段每时刻在训练集的40天中的平均旅行时间
for seg in targets:
    tmp=numpy.array(train[seg])
    mean_mat[seg]=numpy.around(numpy.mean(tmp,axis=0),2)

In [ ]:
numpy.save(r'mean_mat.npy',mean_mat)